In [44]:
import tensorflow as tf

import CNN # External class generated for CNN

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from tensorflow.keras.utils import to_categorical

# Disable eager execution
tf.compat.v1.disable_eager_execution()


epochs=100
batch_size=250


eps_FSGM=0.30

# Select the attacker
#White-box: "FGSM", "BIM", "PGD", 
#Black-box: "SA","ZA"
attack_type = "PGD"
dataset="Fashion_MNIST"   # MNIST, CIRFAR10

#path='results/MNIST_BA.txt' #this for MNIST images
path='C:/Users/02729/OneDrive - Universiti Teknikal Malaysia Melaka/Desktop/IEEE Access paper/Fashion_MNIST/results1/MNIST/MNIST_FGSM.txt' 

In [2]:
# Load and preprocess the MNIST dataset
from art.utils import load_mnist
from tensorflow.keras.datasets import fashion_mnist

#(train_images, train_labels), (test_images, test_labels), min_pixel_value, max_pixel_value = load_mnist()
#train_images, test_images = train_images / max_pixel_value, test_images / max_pixel_value

# Load the Fashion MNIST dataset
# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocess the data
train_images = train_images.reshape((60000, 28, 28, 1)) / 255.0
test_images = test_images.reshape((10000, 28, 28, 1)) / 255.0

min_pixel_value = 0.0
max_pixel_value = 1.0


#print(train_images.shape, one_hot_labels)
classifier, t_consumed = CNN.train(train_images, train_labels, epochs, batch_size, min_pixel_value, max_pixel_value)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 25s 421us/sample - loss: 0.7886 - accuracy: 0.7108
Epoch 2/10
60000/60000 [==============================] - 24s 393us/sample - loss: 0.4478 - accuracy: 0.8371
Epoch 3/10
60000/60000 [==============================] - 23s 385us/sample - loss: 0.3817 - accuracy: 0.8636
Epoch 4/10
60000/60000 [==============================] - 22s 367us/sample - loss: 0.3416 - accuracy: 0.8768
Epoch 5/10
60000/60000 [==============================] - 22s 369us/sample - loss: 0.3137 - accuracy: 0.8859
Epoch 6/10
60000/60000 [==============================] - 22s 370us/sample - loss: 0.2916 - accuracy: 0.8948
Epoch 7/10
60000/60000 [==============================] - 22s 364us/sample - loss: 0.2771 - accuracy: 0.9008
Epoch 8/10
60000/60000 [==============================] - 22s 369us/sample - loss: 0.2674 - accuracy: 0.9027
Epoch 9/10
60000/60000 [==============================] - 22s 363us/sample - loss: 0.2488 - accuracy: 0.9

In [3]:
test_preds = classifier.predict(test_images)
test_labels_pred = np.argmax(test_preds, axis=1)
accuracy_score(test_labels, test_labels_pred)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


0.8979

In [39]:
#Evaluate the model
acc, prc, rec, f1 = CNN.test(classifier, test_images, test_labels)

#write the results
import Writer
results ="===========\n" +  dataset + "-\n\nRegular:\n acc, prc, rec, f1, epochs, batch size, time: acc, prc, rec, f1\n"
results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(epochs)+ "," + str(batch_size) + "," + str(int(t_consumed)) +"\n" 
Writer.write_results(results, path)


acc, prc, rec, f1: 89.79 89.83 89.79 89.74


In [40]:
# Extract the features for the regular images
from tensorflow.keras import models

model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_2').output)
Features_regular = model_output.predict(test_images)
print(Features_regular.shape)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(10000, 32)


In [41]:
# Generate adversarial examples using the FastGradientMethod attack
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent #white-box
from art.attacks.evasion import ZooAttack, SquareAttack, BoundaryAttack #black-box


# Initialize the attack
attack = None

# Select the attack
if attack_type == "FGSM":
    attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)
if attack_type == "BIM":
    attack = BasicIterativeMethod(estimator=classifier, eps=eps_FSGM)
elif attack_type == "PGD":
    attack = ProjectedGradientDescent(estimator=classifier, eps=eps_FSGM)
elif attack_type == "SA":
    attack = SquareAttack(estimator=classifier,eps=eps_FSGM)
elif attack_type == "ZA":
    attack = ZooAttack(classifier=classifier, max_iter=2, learning_rate=eps_FSGM)



adv_images = attack.generate(test_images)


PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [42]:
# Evaluate the model on the adversarial test set images
acc, prc, rec, f1 = CNN.test(classifier, adv_images, test_labels)

#write the results
results =dataset + "-Adversarial:\n acc, prc, rec, f1, epochs, attack_type, eps_BIM\n"
results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + attack_type + "," + str(eps_FSGM) +"\n" 
Writer.write_results(results, path)

acc, prc, rec, f1: 6.79 7.44 6.79 6.27


In [25]:
# Extract the features for the adversarial images
model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_2').output)
Features_adversarial = model_output.predict(adv_images)
print(Features_adversarial.shape)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(10000, 32)


In [26]:
# merge regular and feature test images
from sklearn.model_selection import train_test_split

#merge data features of regular and adversarial images
reg_adv_images = np.concatenate((Features_regular, Features_adversarial), axis=0)

# generate labels for regular and adversarial, and merge them

label_reg = np.zeros((10000, 1))
label_adv = np.ones((10000, 1))
reg_adv_labels=np.concatenate((label_reg, label_adv), axis=0)

print("data details: ", reg_adv_images.shape, " , labels: ", reg_adv_labels.shape)

X_train, X_test, y_train, y_test = train_test_split(reg_adv_images, reg_adv_labels, test_size=0.20, random_state=42)

#fractin to use a part of the merged regular and adversarial images
fraction=1
#reduce the length because the memory is out
X_train=X_train[:int(X_train.shape[0]/fraction)]
X_test=X_test[:(int(X_test.shape[0]/fraction))]
y_train=y_train[:(int(y_train.shape[0]/fraction))]
y_test=y_test[:(int(y_test.shape[0]/fraction))]
X_train, X_test, y_train, y_test
print("data after split : X_train, X_test, y_train, y_test\n", X_train.shape, X_test.shape, y_train.shape, y_test.shape)


data details:  (20000, 32)  , labels:  (20000, 1)
data after split : X_train, X_test, y_train, y_test
 (16000, 32) (4000, 32) (16000, 1) (4000, 1)


In [27]:
'''import ML_Classifiers as MLC

RF_Pred= MLC.Random_Forest(X_train, X_test, y_train, y_test)
DT_Pred= MLC.DT(X_train, X_test, y_train, y_test)
KNN_Pred= MLC.KNN(X_train, X_test, y_train, y_test)
XGB_Pred= MLC.XGB(X_train, X_test, y_train, y_test)
NN_Pred= MLC.NN(X_train, X_test, y_train, y_test)
NN_Pred= MLC.GBM(X_train, X_test, y_train, y_test)'''


'import ML_Classifiers as MLC\n\nRF_Pred= MLC.Random_Forest(X_train, X_test, y_train, y_test)\nDT_Pred= MLC.DT(X_train, X_test, y_train, y_test)\nKNN_Pred= MLC.KNN(X_train, X_test, y_train, y_test)\nXGB_Pred= MLC.XGB(X_train, X_test, y_train, y_test)\nNN_Pred= MLC.NN(X_train, X_test, y_train, y_test)\nNN_Pred= MLC.GBM(X_train, X_test, y_train, y_test)'

In [28]:
# integrate the adversarial detector model to CNN to exclude adversarial images or outlier images.

size_advs=10000
test_reg_adv_images=np.concatenate((test_images,adv_images[:size_advs]), axis=0)
test_reg_adv_labels=np.concatenate((test_labels,test_labels[:size_advs]), axis=0)
print(test_reg_adv_images.shape, test_reg_adv_labels.shape)

# Shuffle the data
indices = np.arange(len(test_reg_adv_images))
np.random.shuffle(indices)
test_reg_adv_images= test_reg_adv_images[indices]
test_reg_adv_labels = test_reg_adv_labels[indices]

# Evaluate the model after integrate regular and adversarial images
acc, prc, rec, f1 = CNN.test(classifier, test_reg_adv_images, test_reg_adv_labels)
results =dataset + "-Regular+Adversarial:\n acc, prc, rec, f1\n"
results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) +"\n" 
Writer.write_results(results, path)


(20000, 28, 28, 1) (20000,)
acc, prc, rec, f1: 48.16 56.26 48.16 50.36


In [29]:
# Extract features 
model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_2').output)
Features_adv_reg = model_output.predict(test_reg_adv_images)
print(Features_adversarial.shape)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(10000, 32)


In [30]:
#to ensure that the size of regular and adversarial image size are the same to do fair evaluation
def make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels):
    diff=int(np.absolute(clean_images.shape[0] - attacked_images.shape[0])/2)
    size=int((clean_images.shape[0]+attacked_images.shape[0])/2)
    temp_clean_images=np.ones((size,28,28,1))
    temp_attacked_images=np.ones((size,28,28,1))

    temp_clean_labels=np.ones((size,10))
    temp_attacked_labels=np.ones((size,10))
    
    if(clean_images.shape[0]>attacked_images.shape[0]):
        temp_clean_images=clean_images[:size]
        temp_clean_labels=clean_labels[:size]
        
        temp_attacked_images=np.concatenate((attacked_images,clean_images[-diff:]), axis=0)
        temp_attacked_labels=np.concatenate((temp_attacked_labels,clean_labels[-diff:]), axis=0)
        
    elif(clean_images.shape[0]<attacked_images.shape[0]):
        temp_attacked_images=attacked_images[:size]
        temp_attacked_labels=attacked_labels[:size]

        temp_clean_images=np.concatenate((clean_images,attacked_images[-diff:]), axis=0)
        temp_clean_labels=np.concatenate((clean_labels,attacked_labels[-diff:]), axis=0)

    clean_images=temp_clean_images
    clean_labels=temp_clean_labels
    
    attacked_images=temp_attacked_images
    attacked_labels=temp_attacked_labels
    
    #print("clean_images.shape, attacked_images.shape", clean_images.shape, attacked_images.shape)
    #print("clean_labels.shape, attacked_labels.shape", clean_labels.shape, attacked_labels.shape)
    
    return clean_images,clean_labels, attacked_images, attacked_labels

#Exclude adversarial images from testing and use only the regular images

def retrurn_clean_images(adv_images_pred_binary):
    
    # Count the number of times 1 appears in the array
    number_of_1s = np.count_nonzero(adv_images_pred_binary == 1)
    # Count the number of times 0 appears in the array
    number_of_0s = np.count_nonzero(adv_images_pred_binary == 0)
    print("The number of attacked immages 1 is:", number_of_1s)
    print("The number of clean images 0 is:", number_of_0s)

    #Exclude adversarial images from testing and use only the regular images
    # Create a boolean mask use only the clean images
    mask_clean = np.where(adv_images_pred_binary == 0)[0]
    mask_adv = np.where(adv_images_pred_binary == 1)[0]

    # Exclude adversarial images
    #clean images
    clean_images = test_reg_adv_images[mask_clean]
    clean_labels = test_reg_adv_labels[mask_clean]
    #Adversarial images,attacked images
    attacked_images = test_reg_adv_images[mask_adv]
    attacked_labels = test_reg_adv_labels[mask_adv]
    
    clean_images,clean_labels, attacked_images, attacked_labels = make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels)
    
    print("clean_images:", clean_images.shape)
    print("clean_labels:", clean_labels.shape)
    
    return clean_images, clean_labels


def eval_model_after_exclude_adv_image(model_name, new_test_clean_images, new_test_clean_labels, t_consumed):# Evaluate the model after excluding the adversarial images
    acc, prc, rec, f1 = CNN.test(classifier, new_test_clean_images, new_test_clean_labels)
    results =model_name + ":\n acc, prc, rec, f1, time_detection\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(int(t_consumed)) + "\n" 
    Writer.write_results(results, path)

In [31]:
#Evaluate the mosdel after inegrate NN as outlier detection to exclude Adversarial images
import ML_Classifiers as MLC
t_str=time.time()

adv_images_pred_binary= MLC.NN(X_train, X_test, y_train, y_test,eps_FSGM, attack_type, Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+NN_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)

Train on 16000 samples
Epoch 1/50
16000/16000 [==============================] - 1s 47us/sample - loss: 0.1936 - accuracy: 0.9274
Epoch 2/50
16000/16000 [==============================] - 1s 39us/sample - loss: 0.1037 - accuracy: 0.9639
Epoch 3/50
16000/16000 [==============================] - 1s 34us/sample - loss: 0.0862 - accuracy: 0.9705
Epoch 4/50
16000/16000 [==============================] - 1s 35us/sample - loss: 0.0774 - accuracy: 0.9734
Epoch 5/50
16000/16000 [==============================] - 1s 38us/sample - loss: 0.0691 - accuracy: 0.9761
Epoch 6/50
16000/16000 [==============================] - 1s 38us/sample - loss: 0.0679 - accuracy: 0.9749
Epoch 7/50
16000/16000 [==============================] - 1s 38us/sample - loss: 0.0641 - accuracy: 0.9776
Epoch 8/50
16000/16000 [==============================] - 1s 38us/sample - loss: 0.0602 - accuracy: 0.9791
Epoch 9/50
16000/16000 [==============================] - 1s 39us/sample - loss: 0.0589 - accuracy: 0.9791
Epoch 10/50
16

C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:251: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


The number of attacked immages 1 is: 10074
The number of clean images 0 is: 9926
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)
acc, prc, rec, f1: 89.11 89.21 89.11 89.09


In [32]:
##Evaluate the mosdel after inegrate RF as outlier detection to exclude Adversarial images
import ML_Classifiers as MLC
t_str=time.time()

adv_images_pred_binary= MLC.Random_Forest(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+RF_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf_rf.fit(X_train, y_train)


RF detection: acc, prc, rec, f1: 97.05 97.07 97.05 97.05


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:38: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


The number of attacked immages 1 is: 10040
The number of clean images 0 is: 9960
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)
acc, prc, rec, f1: 89.33 89.4 89.33 89.29


In [33]:
##Evaluate the mosdel after inegrate DT as outlier detection to exclude Adversarial images
t_str=time.time()
adv_images_pred_binary= MLC.DT(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+DT_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)

DT detection: acc, prc, rec, f1: 94.65 94.65 94.65 94.65
The number of attacked immages 1 is: 10020
The number of clean images 0 is: 9980
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:113: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 88.95 89.06 88.95 88.94


In [34]:
##Evaluate the mosdel after inegrate KNN as outlier detection to exclude Adversarial images
t_str=time.time()
adv_images_pred_binary= MLC.KNN(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+KNN_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNN detection: acc, prc, rec, f1: 97.32 97.34 97.32 97.33


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


The number of attacked immages 1 is: 10108
The number of clean images 0 is: 9892
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)
acc, prc, rec, f1: 88.43 88.59 88.43 88.44


In [35]:
##Evaluate the mosdel after inegrate GBM as outlier detection to exclude Adversarial images
t_str=time.time()

adv_images_pred_binary= MLC.GBM(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+GBM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBM detection: acc, prc, rec, f1: 96.28 96.3 96.28 96.27
The number of attacked immages 1 is: 10162
The number of clean images 0 is: 9838
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 87.28 87.51 87.28 87.31


In [36]:
##Evaluate the mosdel after inegrate XGB as outlier detection to exclude Adversarial images
t_str=time.time()

adv_images_pred_binary= MLC.XGB(X_train, X_test, y_train, y_test,Features_adv_reg)

new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+XGB_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)

XGB detection: acc, prc, rec, f1: 89.82 89.92 89.82 89.82
The number of attacked immages 1 is: 10400
The number of clean images 0 is: 9600
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\ML_Classifiers.py:160: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 80.58 81.56 80.58 80.94


In [37]:
##Evaluate the mosdel after inegrate FAM as outlier detection to exclude Adversarial images
import FAM 

t_str=time.time()
ArtMap=FAM.train(X_train[:5000], y_train[:5000])

adv_images_pred_binary = FAM.test(X_test[:1000], y_test[:1000], ArtMap, Features_adv_reg)

adv_images_pred_binary=np.array(adv_images_pred_binary)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+FAM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)


73  values were replaced from  1000
FAM detection: acc, prc, rec, f1: 76.3 73.6 80.37 76.83


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\Fashion_MNIST\FAM.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


{'index': 0, 'ArtB': [0.0, 1.0], 'id': '0010'}
The number of attacked immages 1 is: 11028
The number of clean images 0 is: 8972
clean_images: (10000, 28, 28, 1)
clean_labels: (10000,)
acc, prc, rec, f1: 69.45 72.2 69.45 70.48


In [21]:
'''
from art.attacks.evasion import ZooAttack
attack = ZooAttack(classifier=classifier)



#attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)
adv_images = attack.generate(test_images)

attack = SquareAttack(estimator=classifier,eps=eps_FSGM)
adv_images = attack.generate(test_images)

'''

'\nfrom art.attacks.evasion import ZooAttack\nattack = ZooAttack(classifier=classifier)\n\n\n\n#attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)\nadv_images = attack.generate(test_images)\n\nattack = SquareAttack(estimator=classifier,eps=eps_FSGM)\nadv_images = attack.generate(test_images)\n\n'

In [50]:
from scipy.stats import friedmanchisquare
import pandas as pd

# Create a DataFrame with the provided data
data = {
    'Dataset': ['MNIST', 'MNIST', 'CIFAR-10', 'CIFAR-10'],
    'Method': ['FGSM', 'PGD', 'FGSM', 'PGD'],
    'BU': [82.2, 77.8, 84.0, 56.5],
    'FS': [97.96, 97.19, 32.5, 4.2],
    'DNR': [79.67, 59.21, 30.23, 18.23],
    'LID': [77.46, 77.03, 76.15, None],  # Assuming '-' means missing data
    'ANR': [92.8, 74.6, 83.2, 59.2],
    'FAD': [75.3, 73.2, 73.4, 74.4],
    'EPM': [74.3, 71.5, 61.4, 66.4],
    'SFAD': [98.61, 81.0, 80.14, 41.2],
    'MNM': [98.62, 97.91, 84.31, 84.68]
}

df = pd.DataFrame(data)

# Remove 'LID' column for Friedman test (due to missing data)
df = df.drop(columns=['LID'])

# Perform Friedman test
# Assuming 'df' is your DataFrame and columns 'BU', 'FS', 'DNR', 'ANR', 'FAD', 'EPM', 'SFAD', 'MNM' are the methods
results = [df[method].tolist() for method in ['BU', 'FS', 'DNR', 'ANR', 'FAD', 'EPM', 'SFAD', 'MNM']]
statistic, p_value = friedmanchisquare(*results)

print(f"Friedman Test Statistic: {statistic}")
print(f"P-value: {p_value}")

print(p_value)
# Check if the Friedman test is significant (p < 0.05)
if p_value < 0.05:
    # Perform Nemenyi test
    nemenyi_result = posthoc_nemenyi(df_for_test)
    print(nemenyi_result)
else:
    print("Friedman test not significant, post hoc test not performed.")


Friedman Test Statistic: 15.75
P-value: 0.02750116179664552
0.02750116179664552


ValueError: Specify correct column names using `group_col` and `val_col` args

In [49]:
import pandas as pd
from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_nemenyi

# Create a DataFrame with the provided data
data = {
    'Dataset': ['MNIST', 'MNIST', 'CIFAR-10', 'CIFAR-10'],
    'Method': ['FGSM', 'PGD', 'FGSM', 'PGD'],
    'BU': [82.2, 77.8, 84.0, 56.5],
    'FS': [97.96, 97.19, 32.5, 4.2],
    'DNR': [79.67, 59.21, 30.23, 18.23],
    'LID': [77.46, 77.03, 76.15, None],  # Assuming '-' means missing data
    'ANR': [92.8, 74.6, 83.2, 59.2],
    'FAD': [75.3, 73.2, 73.4, 74.4],
    'EPM': [74.3, 71.5, 61.4, 66.4],
    'SFAD': [98.61, 81.0, 80.14, 41.2],
    'MNM': [98.62, 97.91, 84.31, 84.68]
}

df = pd.DataFrame(data)

# Exclude 'Dataset' and 'Method' columns for the test
df_for_test = df.drop(columns=['Dataset', 'Method'])

# Perform Friedman test
statistic, p_value = friedmanchisquare(*df_for_test.values.T)
print(p_value)
# Check if the Friedman test is significant (p < 0.05)
if p_value < 0.05:
    # Perform Nemenyi test
    nemenyi_result = posthoc_nemenyi(df_for_test)
    print(nemenyi_result)
else:
    print("Friedman test not significant, post hoc test not performed.")


nan
Friedman test not significant, post hoc test not performed.
